In [1]:
from scienceworld import ScienceWorldEnv
from sources.agent import BDIAgent
from sources.scienceworld import parse_observation, load_step_function

from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [2]:
#hg_model_hub_name = "alisawuffles/roberta-large-wanli"
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
nli_model = NLIModel(hg_model_hub_name, device='cuda')

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model size: 355,362,819


In [24]:
# MAIN GOAL


melt_plan = """
    IF your goal is to melt mercury THEN
        move to kitchen,
        pick up thermometer,
        get metal pot,
        get mercury,
        focus on substance in metal pot,
        freeze mercury,
        heat mercury on blast furnace,
        focus on substance in metal pot
"""

# get metal pot
subplan_a = """
    IF your goal is to get metal pot CONSIDERING you are in the kitchen AND you see a cupboard THEN
        open cupboard,
        pick up metal pot
"""


subplan_b = """
    IF your goal is to get mercury CONSIDERING you are not in workshop AND you have metal pot in your inventory THEN
        move to workshop,
        pour glass cup into metal pot in inventory
"""

subplan_c = """
    IF your goal is to freeze mercury CONSIDERING you have a metal pot with mercury in your inventory THEN
        move to workshop,
        open freezer,
        move metal pot to freezer,
        wait,
        wait,
        use thermometer on metal pot,
        focus on substance in metal pot,
        pick up metal pot
"""

# heat water on stove
subplan_d = """
    IF your goal is to heat mercury on blast furnace CONSIDERING you have metal pot with mercury in your inventory THEN
        move to foundry,
        open blast furnace,
        move metal pot to blast furnace,
        activate blast furnace,
        use thermometer on metal pot,
        focus on substance in metal pot,
        wait,
        use thermometer on metal pot
"""

all_plans = [melt_plan, subplan_a, subplan_b, subplan_c, subplan_d]

pl = PlanLibrary()
#pl.load_plans_from_strings(all_plans)  # load plans from strings above
#pl.load_plans_from_file("plans_navigation.txt")  # load plans from file
print(pl.plans.keys())

dict_keys(['find a(n) non-living thing by moving it to the orange box in the living room', 'find a(n) non-living thing by moving it to the yellow box in the living room', 'find a(n) non-living thing by moving it to the purple box in the living room', 'find a(n) non-living thing by moving it to the red box in the bedroom', 'find a(n) non-living thing by moving it to the green box in the bedroom', 'find a(n) non-living thing by moving it to the blue box in the bedroom', 'find a(n) non-living thing by moving it to the orange box in the bedroom', 'find a(n) non-living thing by moving it to the yellow box in the bedroom', 'find a(n) non-living thing by moving it to the purple box in the bedroom', 'find a(n) non-living thing by moving it to the red box in the workshop', 'find a(n) non-living thing by moving it to the green box in the workshop', 'find a(n) non-living thing by moving it to the blue box in the workshop', 'find a(n) non-living thing by moving it to the orange box in the workshop

In [25]:
lengths = [len(plan) for key, plan in pl.plans.items()]
sum(lengths)

23

In [70]:
env = ScienceWorldEnv("", "", envStepLimit=100)

#root_event = 'use chemistry to create green paint'

task = 'melt'
env.load(task, 0)
#randVariationIdx = env.getRandomVariationTest()
randVariationIdx = 24
env.load(task, randVariationIdx)

goal = env.getTaskDescription().split('.')[0].replace("Your task is to", "").strip()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=goal, valid_actions=info['valid'])
print(current_state.look)
goal

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Task Name: boil variation 24
Task Description: Your task is to melt mercury. First, focus on the substance. Then, take actions that will cause it to change its state of matter.
['This room is called the bathroom.', 'You see the agent', 'You see a substance called air', 'You see a bathtub, which is turned off. In the bathtub is: nothing.', 'You see a drawing', 'You see a glass cup (containing nothing)', 'You see a sink, which is turned off. In the sink is: nothing.', 'You see a toilet. In the toilet is: A drain, which is open, a substance called water.', 'A door to the kitchen (that is closed)']


'melt mercury'

In [71]:
step_function = load_step_function(env, goal=goal)
agent = BDIAgent(plan_library=pl, nli_model=nli_model)
last_state = agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())
print(last_state.reward)

Action: open door -> Obs: The door is now open. -> score 0
Action: go to kitchen -> Obs: You move to the kitchen. -> score 0
Action: pick up thermometer -> Obs: You move the thermometer to the inventory. -> score 0
Action: open cupboard -> Obs: The cupboard is now open. -> score 0
Action: pick up metal pot -> Obs: You move the metal pot to the inventory. -> score 0
Action: open door to hallway -> Obs: The door is now open. -> score 0
Action: go to hallway -> Obs: You move to the hallway. -> score 0
Action: open door to workshop -> Obs: The door is now open. -> score 0
Action: go to workshop -> Obs: You move to the workshop. -> score 3
Action: pour glass cup into metal pot in inventory -> Obs: No known action matches that input. -> score 3
Action: focus on substance in metal pot -> Obs: No known action matches that input. -> score 3
No plan found for event (freeze mercury) with beliefs (['In your inventory, you see: a metal pot (containing nothing) an orange a thermometer, currently rea

In [6]:
last_state.look

['This room is called the foundry.',
 'You see the agent',
 'You see a substance called air',
 'You see a blast furnace, which is turned off. The blast furnace door is closed.',
 'You see a sink, which is turned off. In the sink is: nothing.',
 'You see a table. On the table is: nothing.',
 'A door to the outside (that is open)']